# Ramsey Numbers

#### Navigation
* [Graphs](./Graphs.ipynb)
* [Graphset Enumeration](./GraphsetEnumeration.ipynb)
* [Subgraph Isomorphism - Subgraph Search](./SubgraphIsomorphism.ipynb)

### Useful References

* [Ramsey's Theorem](https://en.wikipedia.org/wiki/Ramsey%27s_theorem)

### Problem

In [2]:
import time
from queue import LifoQueue
from copy import deepcopy

from graph import Graph

import polars as pl

In [3]:
frame_schema = {
    "n": pl.UInt64,
    "F": pl.Object,
    "H": pl.Object,
    "expected_result": pl.Boolean,
    "linear_full_result": pl.Boolean,
    "tree_reduced_enumeration_result": pl.Boolean,
    "linear_full_time_ms": pl.Float64,
    "tree_reduced_enumeration_time_ms": pl.Float64
}

data = []
for n in range(3, 6):
    data.append([ n, Graph.complete_n(2), Graph.empty_n(2), True, None, None, 0., 0. ])
    data.append([ n, Graph.complete_n(3), Graph.empty_n(3), n >= 6, None, None, 0., 0. ])
    data.append([ n, Graph.complete_n(3), Graph.empty_n(4), n >= 9, None, None, 0., 0. ])
    data.append([ n, Graph.complete_n(3), Graph.empty_n(5), n >= 14, None, None, 0., 0. ])

upper_ramsey_data = pl.DataFrame(
    data=data,
    schema=frame_schema,
    orient="row"
)

upper_ramsey_data

n,F,H,expected_result,linear_full_result,tree_reduced_enumeration_result,linear_full_time_ms,tree_reduced_enumeration_time_ms
u64,object,object,bool,bool,bool,f64,f64
3,1,0,true,null,null,0.0,0.0
3,111,000,false,null,null,0.0,0.0
3,111,000000,false,null,null,0.0,0.0
3,111,0000000000,false,null,null,0.0,0.0
4,1,0,true,null,null,0.0,0.0
…,…,…,…,…,…,…,…
4,111,0000000000,false,null,null,0.0,0.0
5,1,0,true,null,null,0.0,0.0
5,111,000,false,null,null,0.0,0.0


In [4]:
def subgraph_search(G: Graph, H: Graph):
    """
        Decide if H is in G
            * return True if H in G
            * return False if H not in G
    """

    stack = LifoQueue()
    stack.put([])

    while not stack.empty():
        item = stack.get()

        if len(item) == H.properties.order:
            local_result = True
            for vert_a_H, vert_a_G in enumerate(item):
                for vert_b_H, vert_b_G in enumerate(item[vert_a_H + 1:]):
                    vert_b_H += vert_a_H + 1
                    if vert_a_G == vert_b_G or H[vert_a_H][vert_b_H] == -1:
                        continue

                    local_result = local_result and (G[vert_a_G][vert_b_G] == H[vert_a_H][vert_b_H])

            if local_result:
                return True
        else:
            for i in range((item[-1] + 1) if len(item) > 0 else 0, G.properties.order):
                new_item = deepcopy(item)
                new_item.append(i)
                stack.put(new_item)

    return False

In [5]:
def is_upper_ramsey_linear_full(n: int, F: Graph, H: Graph):
    result = True
    for i in range(0, int(2**(n*(n-1)/2))):
        G = Graph.from_id(i, n)
        result = result and (subgraph_search(G, F) or subgraph_search(G, H))

    return result

In [6]:
def is_upper_ramsey_tree_reduced_enumeration(n: int, F: Graph, H: Graph):
    stack = LifoQueue()
    result = True

    for i in range(n):
        new_item = []
        for _ in range(i):
            new_item.append("1")
        for _ in range(n - 1 - i):
            new_item.append("0")

        stack.put(new_item)

    while not stack.empty():
        item = stack.get()

        if len(item) < int(n*(n-1)/2):
            item_left_child = deepcopy(item)
            item_left_child.append('0')

            item_right_child = deepcopy(item)
            item_right_child.append('1')

            stack.put(item_right_child)
            stack.put(item_left_child)

        else:
            graph_id = int("".join(item), base=2)
            G = Graph.from_id(graph_id, n)
            result = result and (subgraph_search(G, F) or subgraph_search(G, H))

    return result


In [7]:
def is_upper_ramsey_tree_search(n: int, F: Graph, H: Graph):
    stack = LifoQueue()
    result = True

    for i in range(n):
        new_item = []
        for _ in range(i):
            new_item.append("1")
        for _ in range(n - 1 - i):
            new_item.append("0")

        stack.put(new_item)

    while not stack.empty():
        item = stack.get()

        if len(item) < int(n*(n-1)/2):
            item_left_child = deepcopy(item)
            item_left_child.append('0')

            item_right_child = deepcopy(item)
            item_right_child.append('1')

            right_graph_id = int("".join(item_right_child), base=2)
            right_G = Graph.from_id(right_graph_id, n)
            print(right_G)
            if not (subgraph_search(right_G, F) or subgraph_search(right_G, H)):
                stack.put(item_right_child)

            left_graph_id = int("".join(item_left_child), base=2)
            left_G = Graph.from_id(left_graph_id, n)
            if not (subgraph_search(left_G, F) or subgraph_search(left_G, H)):
                stack.put(item_left_child)

        else:
            graph_id = int("".join(item), base=2)
            G = Graph.from_id(graph_id, n)
            result = result and (subgraph_search(G, F) or subgraph_search(G, H))

    return result

In [8]:
# Testing Brute Force
test_data = []
for row in upper_ramsey_data.iter_rows(named=True):
    start_time = time.perf_counter()
    row['linear_full_result'] = is_upper_ramsey_linear_full(row['n'], row['F'], row['H'])
    end_time = time.perf_counter()
    row['linear_full_time_ms'] = (end_time - start_time) * 1000
    test_data.append(row)

upper_ramsey_data = pl.DataFrame(
    data=test_data,
    schema=frame_schema,
    orient="row"
)

In [9]:
# Testing Optimised A
test_data = []
for row in upper_ramsey_data.iter_rows(named=True):
    start_time = time.perf_counter()
    row['tree_reduced_enumeration_result'] = is_upper_ramsey_tree_reduced_enumeration(row['n'], row['F'], row['H'])
    end_time = time.perf_counter()
    row['tree_reduced_enumeration_time_ms'] = (end_time - start_time) * 1000
    test_data.append(row)

upper_ramsey_data = pl.DataFrame(
    data=test_data,
    schema=frame_schema,
    orient="row"
)

In [11]:
# Testing Optimised B
test_data = []
for row in upper_ramsey_data.iter_rows(named=True):
    start_time = time.perf_counter()
    # row['tree_reduced_enumeration_result'] = is_upper_ramsey_tree_search(row['n'], row['F'], row['H'])
    print(is_upper_ramsey_tree_search(row['n'], row['F'], row['H']))
    end_time = time.perf_counter()
    # row['tree_reduced_enumeration_time_ms'] = (end_time - start_time) * 1000
    # test_data.append(row)

# upper_ramsey_data = pl.DataFrame(
#     data=test_data,
#     schema=frame_schema,
#     orient="row"
# )

AttributeError: 'int' object has no attribute 'properties'

In [10]:
# Convert subgraph objects to string
new_F = upper_ramsey_data.select(["F"]).get_columns()[0].map_elements(lambda G: G.properties.id, return_dtype=pl.UInt64)
new_H = upper_ramsey_data.select(["H"]).get_columns()[0].map_elements(lambda G: G.properties.id, return_dtype=pl.UInt64)
upper_ramsey_data = upper_ramsey_data.with_columns(F = new_F)
upper_ramsey_data = upper_ramsey_data.with_columns(H = new_H)

upper_ramsey_data.write_csv("./data/upper_ramsey.csv")
upper_ramsey_data

n,F,H,expected_result,linear_full_result,tree_reduced_enumeration_result,linear_full_time_ms,tree_reduced_enumeration_time_ms
u64,u64,u64,bool,bool,bool,f64,f64
3,1,0,true,true,true,0.970458,0.328667
3,7,0,false,false,false,0.468792,0.185041
3,7,0,false,false,false,0.252042,0.176208
3,7,0,false,false,false,0.192875,0.173167
4,1,0,true,true,true,2.536958,3.03175
…,…,…,…,…,…,…,…
4,7,0,false,false,false,1.443083,1.294125
5,1,0,true,true,true,51.092833,18.263167
5,7,0,false,false,false,54.006875,17.460708
